In [9]:
import copy
import re
import unidecode
import csv

import numpy as np 
import pandas as pd

from nltk.corpus import stopwords
from nltk.stem import RSLPStemmer
import string
from gensim.models.phrases import Phraser, Phrases
from gensim.models.word2vec import Word2Vec

#bokeh plotting
from bokeh.io import output_notebook
from bokeh.plotting import show, figure

import spacy
#!python -m spacy download pt_core_news_lg
nlp = spacy.load('pt_core_news_lg')

from nltk.tokenize import word_tokenize
from nltk.tokenize import MWETokenizer


In [10]:
original_csv = '../dados/amostra.sample.1.com_texto.sem_header.csv'

ALL_DATA = True

df = pd.read_csv(original_csv)

if ALL_DATA:
    for sample in range(2, 10 + 1):
        original_csv = '../dados/amostra.sample.{}.com_texto.sem_header.csv'.format(sample)
        df_adicional = pd.read_csv(original_csv)
        df = pd.concat([df, df_adicional], ignore_index=True)

#df = df.sample(n=200, random_state=42, ignore_index=True)
df.shape

(70200, 18)

In [11]:
# Read a text file with special stop words
with open('stopwords.txt', 'r') as f:
    str_f = f.read()
    stopwords_file = set(str_f.split('\n'))
all_stopwords = stopwords.words('portuguese') + list(stopwords_file)

def remove_stop_words(texto):
    sem_stop = [w for w in texto.split() if w not in all_stopwords]
    return " ".join(sem_stop)

def custom_tokenizer(texto):
    mwtokenizer = MWETokenizer()
    mwtokenizer.add_mwe(('#', 'mail'))
    mwtokenizer.add_mwe(('#', 'weblink'))
    mwtokenizer.add_mwe(('#', 'phone'))
    mwtokenizer.add_mwe(('#', 'processo'))
    mwtokenizer.add_mwe(('#', 'currency'))
    mwtokenizer.add_mwe(('#', 'cep'))
    mwtokenizer.add_mwe(('#', 'date'))
    mwtokenizer.add_mwe(('#', 'ordinal'))
    mwtokenizer.add_mwe(('#', 'number'))
    mwtokenizer.add_mwe(('boa', 'vista'))
    mwtokenizer.add_mwe(('boa', 'vista/rr'))
    mwtokenizer.add_mwe(('boa', 'viagem'))
    mwtokenizer.add_mwe(('vara', 'civel'))
    return mwtokenizer.tokenize(word_tokenize(texto, language='portuguese'))

def stemm(texto):
    stemmer = RSLPStemmer()
    return " ".join([str(stemmer.stem(p)) for p in texto.split()])

def lemmatize(texto):
    result = " ".join([w.lemma_ for w in nlp(texto)])
    result=re.sub(" # _ ", ' #_', result)
    result=re.sub('\s{2,}', ' ', result) #removendo espaços duplicados
    return result.lower()

def preprocessor_OLD(texto, replace_email=True, replace_url=True, replace_numbers=True, to_lower=True, replace_stop_words=True, replace_accents=True):
    email_replacer = ' mail### '
    url_replacer = ' weblink### '
    number_replacer = ' ### '

    result = copy.deepcopy(texto)
    result = str(result)
    
    result=result.lower() if to_lower else result
    result=result.replace('{html}',"") 
    
    result=re.sub(r"[a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+", email_replacer, result) if replace_email else result #subtituindo valores de email por 'mail###'
    result=re.sub(r"http\S+", url_replacer, result) if replace_url else result #subtituindo urls por 'weblink###'
    result=re.sub('\d', number_replacer, result) if replace_numbers else result #subtituindo números por '###'
    
    result=re.sub('([.,!?])', r' \1 ', result)  #colocando espaço entre pontuações de palavras
    
    # result=result.replace("<br />", "")
    result=result.replace("!", " ")
    result=result.replace('"', " ")
    result=result.replace("“", " ")
    result=result.replace("”", " ")
    result=result.replace("$", " ")
    result=result.replace("%", " ")
    result=result.replace("&", " ")
    result=result.replace("'", " ")
    result=result.replace("(", " ")
    result=result.replace(")", " ")
    result=result.replace("*", " ")
    result=result.replace("+", " ")
    result=result.replace(",", " ")
    result=result.replace(" - ", " ")
    result=result.replace(" -", " ")
    result=result.replace(" – ", " ")
    result=result.replace(".", " ")
    result=result.replace("/", " ")
    result=result.replace(":", " ")
    result=result.replace(";", " ")
    result=result.replace("<", " ")
    result=result.replace("=", " ")
    result=result.replace(">", " ")
    result=result.replace("?", " ")
    result=result.replace("@", " ")
    result=result.replace("[", " ")
    result=result.replace("\\", " ")
    result=result.replace("]", " ")
    result=result.replace("^", " ")
    result=result.replace(" _ ", " ")
    result=result.replace("`", " ")
    result=result.replace("{", " ")
    result=result.replace("|", " ")
    result=result.replace("}", " ")
    result=result.replace("~", " ")
    
    result=result.replace("§", "")
    result=result.replace("ª", "")
    result=result.replace("º", "")
    result=result.replace("°", "")
    
    # result=re.sub(' +', ' ', result)
    result=unidecode.unidecode(result) if replace_accents else result

    result=result.replace("\n", " ") #removendo quebras de linha
    result=re.sub(r'\s{2,}', ' ', result) #removendo espaços duplicados
    
    result=remove_stop_words(result) if replace_stop_words else result
    result=re.sub(r'\b[a-zA-Z]\s', '', result) #removendo caracteres isolados

    return result

removers = ['!', '\'', '“', '”', '$', ' %', '&', '\'', '(', ')', '*', '+', ',', '-', '–', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '§', 'ª', 'º', '°']

def preprocessor(texto):
    result = copy.deepcopy(texto)
    result = str(result)
    result=result.lower()
    
    email_replacer = ' '
    url_replacer = ' '
    phone_replacer = ' '
    processo_replacer = ' '
    currency_replacer = ' '
    cep_replacer = ' '
    date_replacer = ' '
    ordinal_replacer = ' '
    number_replacer = ' '

    result=re.sub(r"[a-zA-Z0-9._-]+@[a-zA-Z0-9._-]+\.[a-zA-Z0-9_-]+", email_replacer, result) #subtituindo valores de email por coringa
    result=re.sub(r"http\S+", url_replacer, result) #subtituindo urls por coringa
    result=re.sub(r"\d{7}-?\d{2}.?\d{4}.?\d.?\d{2}.?\d{4}", processo_replacer, result) #subtituindo processo por coringa
    result=re.sub(r"([R_r]\$\s{0,}?)(\d{1,3}(\.\d{3})*|\d+)(\,\d{2})?", currency_replacer, result) #subtituindo valor monetário por coringa
    result=re.sub(r"\(\d{2}\)\s9?\d{4}-?\d{4}", phone_replacer, result) #subtituindo telefone por coringa
    result=re.sub(r"(([0-9]{2}\.[0-9]{3}-[0-9]{3})|([0-9]{2}[0-9]{3}-[0-9]{3})|([0-9]{8}))", cep_replacer, result) #subtituindo cep por coringa
    result=re.sub(r"(?:(?:31(\/|-|\.)(?:0?[13578]|1[02]))\1|(?:(?:29|30)(\/|-|\.)(?:0?[1,3-9]|1[0-2])\2))(?:(?:1[6-9]|[2-9]\d)?\d{2})$|(?:29(\/|-|\.)0?2\3(?:(?:(?:1[6-9]|[2-9]\d)?(?:0[48]|[2468][048]|[13579][26])|(?:(?:16|[2468][048]|[3579][26])00))))$|(?:0?[1-9]|1\d|2[0-8])(\/|-|\.)(?:(?:0?[1-9])|(?:1[0-2]))\4(?:(?:1[6-9]|[2-9]\d)?\d{2})$",
        date_replacer, result) #subtituindo data por coringa
    result=re.sub(r'(\d+)(?:ª|a|º|°|\.)', ordinal_replacer, result) #subtituindo números ordinais por coringa
    result=re.sub(r'\s?\d+\s?', number_replacer, result) #subtituindo números por coringa

    result=unidecode.unidecode(result)
    result = custom_tokenizer(result)
    result=[x for x in result if x not in removers] #removendo caracteres isolados

    result=" ".join(result)
    result=result.replace("\n", " ") #removendo quebras de linha
    result=re.sub(r'([[:alnum:]]|_)\1{2,}', ' ', result) #removendo caracteres com 3 ou mais ocorrencias sequenciais
    result=re.sub(r'\s{2,}', ' ', result) #removendo espaços duplicados
    
    result=remove_stop_words(result)
    result=re.sub(r'\b[a-zA-Z]\s', '', result) #removendo caracteres isolados

    return result


In [12]:
df['texto_clean_pure'] = df['texto'].apply(preprocessor)

In [13]:
df['texto_clean_stemmed'] = df['texto_clean_pure'].apply(stemm)

In [14]:
df['texto_clean_lemmatized'] = df['texto_clean_pure'].apply(lemmatize)

In [15]:
#print('ORIGINAL')
#print(df['texto'][0])
print('-'*80)
print('CLEAN PURE')
print(df['texto_clean_pure'][0])
print('-'*80)
print('CLEAN STEMMED')
print(df['texto_clean_stemmed'][0])
print('-'*80)
print('CLEAN LEMMATIZED')
print(df['texto_clean_lemmatized'][0])

--------------------------------------------------------------------------------
CLEAN PURE
processo classe processual cumprimento sentenca assunto principal bancarios valor causa exequente rossinalva ribeiro silva rua ivone pinheiro centro boa_vista/rr executado banco bgn s.a. rua antonio lumack monte edificio empresarial center ii s. /boa_viagem recife/pe cep telefone despacho determino intimacao parte executada manifestar acerca pedido relacao possivel saldo remanescente constante ep prazo quinze dias expedientes necessarios cumpra-se comarca boa_vista rr data constante sistema jarbas lacerda miranda juiz direito titular vara_civel assinado digitalmente
--------------------------------------------------------------------------------
CLEAN STEMMED
process cl process cumpr sentenc assunt princip bancari val caus exequ rossinalv rib silv rua ivon pinh centr boa_vista/rr execut banc bgn s.a. rua antoni lumack mont edifici empresar cent ii s. /boa_vi recife/p cep telefon despach determin

In [16]:
# Remove as linhas sem texto

df['texto_clean_pure'].replace(' ', np.nan, inplace=True)
df.dropna(subset=['texto_clean_pure'], inplace=True)

df.shape

(70200, 21)

### Preprocess the corpus

In [17]:
#text_columns = ['texto_clean_pure', 'texto_clean_stemmed', 'texto_clean_lemmatized']

#for column in text_columns:
  #array_of_words =[]
  #final_corpus = []
  #for content in list(df[column]):
  #  array_of_words.append(content.split())

  #bigrams = Phraser(Phrases(array_of_words, min_count=5, threshold=10))
  ## bigrams.phrasegrams
  #for s in array_of_words:
  #    final_corpus.append(' '.join([word for word in bigrams[s]]))  
  
#  df[column.replace('texto_', 'corpus_')] = df[column]
  #print(final_corpus[0])

In [18]:
#df.head()

### Run word2vec

In [19]:
# word2vec_model = Word2Vec(sentences=final_corpus, vector_size=64,
#                     sg=1, window=10, epochs=5,
#                     min_count=10, workers=2)


In [20]:
# top_n_similarity=3
# check_top_words=['processo', 'cumprimento_sentenca', 'assunto_principal', 'dia', 'intimacao', 'parte']

# final_corpus

#for word in check_top_words:
#    print("TOP {} palavras similar a '{}': {}".format(top_n_similarity, word, word2vec_model.wv.most_similar(word, topn=top_n_similarity)))


### Reduce word vector dimensionality with t-SNE

In [ ]:
# from sklearn.manifold import TSNE

# tsne = TSNE(n_components=2, n_iter=1000, random_state=42, init='random', learning_rate=200)
# vectors = np.asarray(word2vec_model.wv.vectors)
# X_2d = tsne.fit_transform(vectors)
# coords_df = pd.DataFrame(X_2d, columns=['x','y'])
# coords_df['token'] = word2vec_model.wv.index_to_key

# # coords_df.to_csv('clean_mandamus_tsne.csv', index=False)
# # coords_df = pd.read_csv('clean_mandamus_tsne.csv')

# _ = coords_df.plot.scatter('x', 'y', figsize=(12,12),
#                         marker='.', s=10, alpha=0.2)

# subset_df = coords_df.sample(n=5000)

# output_notebook()
# p = figure(plot_width=800, plot_height=800)
# _ = p.text(x=subset_df.x, y=subset_df.y, text=subset_df.token)

# show(p)

### Preprocess the class

In [21]:
# Define todos nos NA para Não é Mandado
df['desc_natureza_mandado'] = df['desc_natureza_mandado'].fillna('Não é Mandado')

print(pd.crosstab(index=df['desc_natureza_mandado'], columns="freq_rel", normalize=True).sort_values(by=['freq_rel'], ascending=False))

col_0                               freq_rel
desc_natureza_mandado                       
Não é Mandado                       0.871211
Intimação                           0.041752
Citação                             0.033390
Intimação de Sentença               0.018590
Intimação para Audiência            0.007236
Intimação Despacho                  0.006952
Citação e Intimação para Audiência  0.004772
Penhora e/ou avaliação              0.003148
Notificação                         0.003048
Busca e Apreensão                   0.002949
Alvará de Soltura                   0.002564
Prisão                              0.002550
Liminar                             0.001026
Reintegração de Posse               0.000313
Averbação                           0.000114
Condução                            0.000071
Medida Cautelar                     0.000071
Adjudicação                         0.000057
Citação por Hora Certa              0.000057
Interrogatório                      0.000043
Inquirição

In [26]:
# Faz a unificação das classes comuns para Intimação
df['classificacao'] = df['desc_natureza_mandado'] \
    .replace(['Intimação de Sentença',  'Intimação para Audiência', 'Intimação Despacho',   'Citação e Intimação para Audiência'], 'Intimação') \
    .replace(['Citação por Hora Certa'], 'Citação') \
    .replace(['Adjudicação', 'Alvará de Soltura', 'Averbação', 'Busca e Apreensão', 'Busca e Apreensão de Crianças', 'Condução', 'Inquirição de Testemunha', 'Interrogatório', 'Liminar', 'Medida Cautelar', 'Notificação', 'Penhora e/ou avaliação', 'Prisão', 'Reintegração de Posse', 'Anotação', 'Depoimento Pessoal', 'Internação'], 'Outros')
    
print(pd.crosstab(index=df['classificacao'], columns="freq_rel", normalize=True).sort_values(by=['freq_rel'], ascending=False))

col_0          freq_rel
classificacao          
Não é Mandado  0.871211
Intimação      0.079302
Citação        0.033447
Outros         0.016040


In [27]:
df['categoria_binaria'] = df['classificacao']
df.loc[df['categoria_binaria'] != 'Não é Mandado', 'categoria_binaria'] = 1 #'GM'
df.loc[df['categoria_binaria'] == 'Não é Mandado', 'categoria_binaria'] = 0 #'NGM'

print(pd.crosstab(index=df['categoria_binaria'], columns="freq_rel", normalize=False).sort_values(by=['freq_rel'], ascending=False))

col_0              freq_rel
categoria_binaria          
0                     61159
1                      9041


In [28]:
df.head()

,mov_ordem_coddocumento,numero_processo,mov_ordem_datarecebimento,mov_ordem_login,ad_mov_ordem_magistrado_caminho,ad_mov_ordem_magistrado_descricao,cod_vara,nome_vara,competencia_vara,cod_natureza_mandado,...,ad_mov_exped_mandado_descricao,parte_nome,parte_tipo,i,texto,texto_clean_pure,texto_clean_stemmed,texto_clean_lemmatized,classificacao,categoria_binaria
0,18597402,9.182751e+18,2018-01-18 18:17:33.783,jarbas.miranda,/2018/jan/28/f/28f20f8e2c5836646c700ff891f5734...,mero expediente,1001064,4ª Vara Cível,VarCiv,NaN,...,NaN,NaN,NaN,1,\nProcesso: 0918275-13.2009.8.23.0010\nClass...,processo classe processual cumprimento sentenc...,process cl process cumpr sentenc assunt princi...,processo classe processual cumprimento sentenc...,Não é Mandado,0
1,31665675,1.317360e+17,2021-08-05 14:20:52.762,daniel,/2021/ago/22/8/2280a179f0d3f298136c2ba7a0c7cc0...,NaN,6017026,2ª Vara Criminal,VarCrim,10.0,...,Citação,CARLOS EDUARDO CAVALCANTI DE SANTANA,PROMOVIDO,1,\nProc. n.° $processo.getNumeroProcessoForma...,proc n.deg processo.getnumeroprocessoformatado...,proc n.deg processo.getnumeroprocessoformat de...,proc n.deg processo.getnumeroprocessoformatado...,Citação,1
2,23681708,8.189782e+18,2019-06-10 12:06:45.934,paulo,/2019/jun/61/1/611c3b714233f31fc3e0300630473c3...,Despacho,1086073,2ª Vara de Família,VarFam,NaN,...,NaN,NaN,NaN,1,2familia@tjrr.jus.br\n \nProc. n.° 0818978-19...,proc n.deg despacho inventariante apresente pr...,proc n.deg despach inventari apres praz dia ul...,proc n.deg despacho inventariante apresente pr...,Não é Mandado,0
3,30970967,1.350481e+17,2021-06-07 22:27:52.729,daniel,/2021/jun/fd/b/fdb0731c9cc3e8e48095df897ec5168...,Despacho,1086016,1ª Vara de Família,VarFam,NaN,...,NaN,NaN,NaN,1,\nProcesso: 0013504-81.2010.8.23.0010\nClass...,processo classe processual inventario assunto ...,process cl process inventari assunt princip in...,processo classe processual inventario assunto ...,Não é Mandado,0
4,17074336,8.151748e+18,2017-07-19 15:57:23.749,dias.eduardo,/2017/jul/00/e/00ed26bd43ebf8622fd8b90335f30a9...,Despacho,6017022,5ª Vara Cível,VarCiv,NaN,...,NaN,NaN,NaN,1,\nProc. n.° 0815174-77.2017.8.23.0010\n \n \...,proc n.deg despacho intime-se autor derradeira...,proc n.deg despach intime-s autor derr vez pra...,proc n.deg despacho intime-se autor derradeira...,Não é Mandado,0


In [29]:
df.to_csv(f"../dados/amostra.sample.final{'.full' if ALL_DATA else ''}.csv", index=False, encoding='utf-8', quotechar='"',
          quoting=csv.QUOTE_NONNUMERIC, escapechar='\\')